# Using DuckDB for Data Analysis

## Overview

In this exercise, you will walk through the previous data preprocessing workflow using DuckDB.

The goal is to replicate the same steps we performed in polars, but this time using SQL queries in DuckDB. This will give you a chance to see how the same transformations can be expressed in SQL and how DuckDB handles data manipulation.

You will learn how to:

- Load and inspect data
- Identify and handle missing values
- Clean and standardize columns
- Perform basic transformations
- Verify the final dataset


## Import Libraries

We are using the Polars library for data manipulation due to its performance advantages, especially with larger datasets.


In [1]:
import duckdb

## Read the Input Parquet File

First, download the `credit_card_transactions-ibm_v2.parquet` file from [this GitHub link](https://github.com/bdi593/datasets/raw/refs/heads/main/credit-card-transactions-ibm/credit_card_transactions-ibm_v2.parquet?download=) and place it in the same folder as your Jupyter notebook.

The file is about 250 MB, which is about 9 times smaller than the original CSV file, making it much faster to read and process.

:::{tip} Why is Parquet smaller than CSV?

Recall that Parquet is a columnar storage format that uses efficient compression and encoding techniques, while CSV is a plain text format that stores data in a row-wise manner without any compression. This is why Parquet files are often significantly smaller than their CSV counterparts.

You can also compress CSV files using different algorithms (like gzip or bzip2), but even then, Parquet often achieves better compression ratios due to its columnar nature and optimized encoding.

:::


### `duckdb.connect()`

The `duckdb.connect()` function creates a connection to a DuckDB database. If you don't specify a database name, it creates an in-memory database that exists only for the duration of the connection. This is useful for ad-hoc analysis and temporary data manipulation without needing to manage a persistent database file.


In [2]:
con = duckdb.connect()

con.execute(
    """
CREATE OR REPLACE VIEW transactions AS
SELECT *
FROM read_parquet('credit_card_transactions-ibm_v2.parquet');
"""
)

# Peek (like df.head())
con.sql("SELECT * FROM transactions LIMIT 5").df()

,User,Card,Amount,Datetime,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
0,0,0,134.09,2002-09-01 06:21:00,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,5300,None,No
1,0,0,38.48,2002-09-01 06:42:00,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,None,No
2,0,0,120.34,2002-09-02 06:22:00,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,None,No
3,0,0,128.95,2002-09-02 17:45:00,Swipe Transaction,3414527459579106770,Monterey Park,CA,91754.0,5651,None,No
4,0,0,104.71,2002-09-03 06:23:00,Swipe Transaction,5817218446178736267,La Verne,CA,91750.0,5912,None,No


## Exploratory Data Analysis (EDA)


### Describe Schema

To understand the structure of the dataset, we can use the `DESCRIBE` command to get information about the columns, their data types, and whether they contain null values.


In [3]:
con.sql("DESCRIBE transactions;").df()

,column_name,column_type,null,key,default,extra
0,User,BIGINT,YES,None,None,None
1,Card,BIGINT,YES,None,None,None
2,Amount,DOUBLE,YES,None,None,None
3,Datetime,TIMESTAMP,YES,None,None,None
4,Use Chip,VARCHAR,YES,None,None,None
5,Merchant Name,BIGINT,YES,None,None,None
6,Merchant City,VARCHAR,YES,None,None,None
7,Merchant State,VARCHAR,YES,None,None,None
8,Zip,DOUBLE,YES,None,None,None
9,MCC,BIGINT,YES,None,None,None


If you're familiar with the `PRAGMA` introspection command from SQLite or another engine, you can also use `PRAGMA table_info('transactions');` to get similar information about the columns in the `transactions` table.


In [4]:
con.sql("PRAGMA table_info('transactions');").df()

,cid,name,type,notnull,dflt_value,pk
0,0,User,BIGINT,False,None,False
1,1,Card,BIGINT,False,None,False
2,2,Amount,DOUBLE,False,None,False
3,3,Datetime,TIMESTAMP,False,None,False
4,4,Use Chip,VARCHAR,False,None,False
5,5,Merchant Name,BIGINT,False,None,False
6,6,Merchant City,VARCHAR,False,None,False
7,7,Merchant State,VARCHAR,False,None,False
8,8,Zip,DOUBLE,False,None,False
9,9,MCC,BIGINT,False,None,False


Note that the `Datetime` column is `TIMESTAMP` type, which means it contains both date and time information. The `Amount` column is `DOUBLE`, which is a floating-point number, and the `Merchant Name` column is `BIGINT`, which were encoded as integers to avoid confusion with real merchant names. The `Errors?` and `Is Fraud?` columns use `VARCHAR` type, which is a variable-length string.


In [5]:
con.sql('SELECT DISTINCT("Is Fraud?") FROM transactions;').df()

,Is Fraud?
0,No
1,Yes


### Convert `"Is Fraud?"` column to a boolean

The code below performs data type conversion on the `"Amount"` column. It utilizes Polars' expression API to remove "$" and "," characters via `replace_all()`, subsequently casting the cleaned values to `Float64` to overwrite the existing column with numeric data.


In [6]:
con.sql(
    """
CREATE OR REPLACE VIEW transactions_with_fraud_flag AS
SELECT
  * EXCLUDE ("Is Fraud?"),
  CASE
    WHEN "Is Fraud?" = 'Yes' THEN TRUE
    WHEN "Is Fraud?" = 'No'  THEN FALSE
    ELSE NULL
  END AS "Is Fraud?"
FROM transactions;
"""
)

In [7]:
con.sql(
    """
SELECT *
FROM transactions_with_fraud_flag
LIMIT 5;
"""
).df()

,User,Card,Amount,Datetime,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
0,0,0,134.09,2002-09-01 06:21:00,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,5300,None,False
1,0,0,38.48,2002-09-01 06:42:00,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,None,False
2,0,0,120.34,2002-09-02 06:22:00,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,None,False
3,0,0,128.95,2002-09-02 17:45:00,Swipe Transaction,3414527459579106770,Monterey Park,CA,91754.0,5651,None,False
4,0,0,104.71,2002-09-03 06:23:00,Swipe Transaction,5817218446178736267,La Verne,CA,91750.0,5912,None,False


You can verify that the `"Is Fraud?"` column is now a boolean type using the `DESCRIBE` command again.


In [8]:
con.sql("DESCRIBE transactions_with_fraud_flag;").df()

,column_name,column_type,null,key,default,extra
0,User,BIGINT,YES,None,None,None
1,Card,BIGINT,YES,None,None,None
2,Amount,DOUBLE,YES,None,None,None
3,Datetime,TIMESTAMP,YES,None,None,None
4,Use Chip,VARCHAR,YES,None,None,None
5,Merchant Name,BIGINT,YES,None,None,None
6,Merchant City,VARCHAR,YES,None,None,None
7,Merchant State,VARCHAR,YES,None,None,None
8,Zip,DOUBLE,YES,None,None,None
9,MCC,BIGINT,YES,None,None,None


### Payment method frequency analysis

The code below provides a high-level summary of the payment methods used across your credit card dataset. The `.value_counts()` method performs a frequency analysis on the `"Use Chip"` column, identifying every unique entry (Chip, Online, and Swipe transactions).


In [9]:
con.sql(
    """
SELECT
  "Use Chip",
  COUNT(*) AS count
FROM transactions
GROUP BY "Use Chip"
ORDER BY count DESC;
        """
)

┌────────────────────┬──────────┐
│      Use Chip      │  count   │
│      varchar       │  int64   │
├────────────────────┼──────────┤
│ Swipe Transaction  │ 15386082 │
│ Chip Transaction   │  6287598 │
│ Online Transaction │  2713220 │
└────────────────────┴──────────┘

Looking at the numbers, you can see that Swipe Transactions are the dominant payment method in this dataset with over 15 million entries, followed by Chip Transactions at roughly 6.2 million. This breakdown is essential for understanding consumer behavior or detecting potential fraud patterns, as certain types of transactions (like "Online" vs. "Swipe") carry different risk profiles.


### Fraud distribution analysis

The code below shows the frequency of the `"Is Fraud?"` column to determine the distribution of legitimate versus fraudulent transactions within your dataset.


In [10]:
con.sql(
    """
SELECT
  "Is Fraud?",
  COUNT(*) AS count
FROM transactions
GROUP BY "Is Fraud?"
ORDER BY count DESC;
        """
)

┌───────────┬──────────┐
│ Is Fraud? │  count   │
│  varchar  │  int64   │
├───────────┼──────────┤
│ No        │ 24357143 │
│ Yes       │    29757 │
└───────────┴──────────┘

The data reveals that the overwhelming majority of transactions - over 24.3 million - are flagged as "No", while only 29,757 are flagged as "Yes". While the fraudulent cases represent a very small fraction of the total volume (roughly 0.12%), identifying this minority is the primary goal of most credit card analytics. Understanding this ratio is a critical first step before building a machine learning model, as it tells you that you'll need specialized techniques, like oversampling or specific loss functions, to account for the rarity of fraud.


### Flag suspicious transactions based on amount

The code below performs feature engineering by creating three new boolean (`True`/`False`) indicators based on specific transaction characteristics. By using `with_columns()` with a list of expressions, Polars efficiently evaluates these conditions in parallel, adding descriptive flags that make the dataset much easier to filter and analyze. Specifically, it identifies refunds (amounts less than zero), large transactions (amounts exceeding $500), and online activity (where the "Use Chip" status matches "Online Transaction").


In [11]:
con.sql(
    """
SELECT
  *,
  "Amount" < 0 AS is_refund,
  "Amount" > 500 AS large_txn,
  "Use Chip" = 'Online Transaction' AS is_online
FROM transactions;
"""
).df()

,User,Card,Amount,Datetime,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?,is_refund,large_txn,is_online
0,0,0,134.09,2002-09-01 06:21:00,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,5300,None,No,False,False,False
1,0,0,38.48,2002-09-01 06:42:00,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,None,No,False,False,False
2,0,0,120.34,2002-09-02 06:22:00,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,None,No,False,False,False
3,0,0,128.95,2002-09-02 17:45:00,Swipe Transaction,3414527459579106770,Monterey Park,CA,91754.0,5651,None,No,False,False,False
4,0,0,104.71,2002-09-03 06:23:00,Swipe Transaction,5817218446178736267,La Verne,CA,91750.0,5912,None,No,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24386895,1999,1,-54.00,2020-02-27 22:23:00,Chip Transaction,-5162038175624867091,Merrimack,NH,3054.0,5541,None,No,True,False,False
24386896,1999,1,54.00,2020-02-27 22:24:00,Chip Transaction,-5162038175624867091,Merrimack,NH,3054.0,5541,None,No,False,False,False
24386897,1999,1,59.15,2020-02-28 07:43:00,Chip Transaction,2500998799892805156,Merrimack,NH,3054.0,4121,None,No,False,False,False
24386898,1999,1,43.12,2020-02-28 20:10:00,Chip Transaction,2500998799892805156,Merrimack,NH,3054.0,4121,None,No,False,False,False


These "flag" columns can be useful for both exploratory analysis and machine learning. Instead of writing complex filters repeatedly, you can now quickly segment your data. For example, to see if "large transactions" are more likely to be "fraudulent" or to calculate the total volume of "online" versus in-person sales. This step transforms raw data into behavioral features, providing the model or the analyst with clear, binary signals that highlight high-interest events within the transaction stream.


### Filter rows based on conditions

You can also apply a filter to isolate a very specific subset of your data: online refunds. By passing multiple conditions into the `.filter()` method, Polars treats them as a logical AND operation, meaning a transaction will only remain in the resulting DataFrame if it is both a negative value (the `"is_refund"` condition) and was conducted as an "Online Transaction" (the `"is_online"` condition).


In [12]:
con.sql(
    """
SELECT *
FROM transactions
WHERE
  "Amount" < 0
  AND "Use Chip" = 'Online Transaction';
"""
).df()

,User,Card,Amount,Datetime,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
0,0,0,-100.0,2004-12-10 20:45:00,Online Transaction,7501849281341469857,ONLINE,None,NaN,4722,None,No
1,0,0,-393.0,2010-10-08 18:00:00,Online Transaction,333722291367506728,ONLINE,None,NaN,4722,None,No
2,0,0,-443.0,2011-12-19 12:50:00,Online Transaction,333722291367506728,ONLINE,None,NaN,4722,None,No
3,0,0,-473.0,2015-11-20 07:42:00,Online Transaction,-8566951830324093739,ONLINE,None,NaN,3640,None,Yes
4,0,2,-461.0,2017-02-21 08:03:00,Online Transaction,7501849281341469857,ONLINE,None,NaN,4722,None,No
...,...,...,...,...,...,...,...,...,...,...,...,...
13494,1998,0,-441.0,2019-02-27 15:09:00,Online Transaction,3694722044710185708,ONLINE,None,NaN,4722,None,No
13495,1998,0,-354.0,2019-11-27 11:08:00,Online Transaction,3694722044710185708,ONLINE,None,NaN,4722,None,No
13496,1998,0,-122.0,2019-12-29 19:40:00,Online Transaction,3694722044710185708,ONLINE,None,NaN,4722,None,No
13497,1999,1,-419.0,2018-10-24 01:48:00,Online Transaction,7501849281341469857,ONLINE,None,NaN,4722,None,No


### Per-user aggregation

The code below performs a user-level aggregation, collapsing millions of individual transaction records into a concise summary of spending behavior for each unique customer. By using `group_by("User")`, you are instructing Polars to organize the data into buckets based on the individual user ID. The `.agg()` function then calculates three key metrics for each bucket:

1. the total volume of money spent,
2. the average cost per purchase, and
3. the total count of transactions (using `pl.len()`).


In [13]:
user_stats = con.sql(
    """
SELECT
  "User",
  SUM("Amount") AS total_spent,
  AVG("Amount") AS average_amount,
  COUNT(*)      AS num_transactions
FROM transactions
GROUP BY "User";
"""
).df()

user_stats.head()

,User,total_spent,average_amount,num_transactions
0,0,1622991.69,81.299989,19963
1,1,723491.89,81.118050,8919
2,2,1475933.36,35.159687,41978
3,3,1186497.51,117.277603,10117
4,4,1798790.91,97.011698,18542


Rolling / window features (SQL-like mental model):


### 10 most recent transactions rolling sum

You can also compute rolling aggregates to capture recent behavior. The code below calculates a rolling sum of the `"Amount"` column over the last 10 transactions for each user. By using `.over("User")`, you ensure that the rolling calculation is performed separately for each individual customer, maintaining the integrity of user-specific spending patterns. The `.rolling_sum(window_size=10)` function then computes the sum of the last 10 transaction amounts, providing insight into recent spending trends.


In [ ]:
df_rolling = con.sql(
    """
SELECT
  *,
  SUM("Amount") OVER (
    PARTITION BY "User"
    ORDER BY "Datetime"
    ROWS BETWEEN 9 PRECEDING AND CURRENT ROW
  ) AS rolling_amount_sum_10
FROM transactions
ORDER BY "User", "Datetime";
"""
).df()

df_rolling.head()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

## Conclusion

You have replicated the data preprocessing pipeline using DuckDB. Then, you performed feature engineering and aggregation to extract meaningful insights!
